In [9]:
import os
from pyTigerGraph import TigerGraphConnection
from dotenv import load_dotenv

load_dotenv()
# We first create a connection to the database
host = os.environ["HOST"]
username = os.getenv("USERNAME", "tigergraph")
password = os.getenv("PASS", "tigergraph")
conn = TigerGraphConnection(
    host=host,
    username=username,
    password=password,
)

In [10]:
conn.graphname = "GraphRAG_pytgdocs"
conn.gsql("""CREATE GRAPH GraphRAG_pytgdocs()""")

'The graph GraphRAG_pytgdocs is created.'

In [11]:
_ = conn.getToken()

In [12]:
# And then add CoPilot's address to the connection. This address
# is the host's address where the CoPilot container is running.
conn.ai.configureCoPilotHost("http://localhost:8000")
conn.ai.initializeSupportAI()

{'host_name': 'https://algotesting.i.tgcloud.io',
 'schema_creation_status': '"Using graph \'GraphRAG_pytgdocs\'\\nSuccessfully created schema change jobs: [add_supportai_schema].\\nWARNING: When modifying the graph schema, reinstalling all affected queries is required, and the duration of this process may vary based on the number and complexity of the queries. To skip query reinstallation, you can run with the \'-N\' option, but manual reinstallation of queries will be necessary afterwards.\\nKick off schema change job add_supportai_schema\\nDoing schema change on graph \'GraphRAG_pytgdocs\' (current version: 0)\\nTrying to add local vertex \'DocumentChunk\' to the graph \'GraphRAG_pytgdocs\'.\\nTrying to add local vertex \'Document\' to the graph \'GraphRAG_pytgdocs\'.\\nTrying to add local vertex \'Concept\' to the graph \'GraphRAG_pytgdocs\'.\\nTrying to add local vertex \'Entity\' to the graph \'GraphRAG_pytgdocs\'.\\nTrying to add local vertex \'Relationship\' to the graph \'Grap

In [13]:
access = os.environ["AWS_ACCESS_KEY_ID"]
sec = os.environ["AWS_SECRET_ACCESS_KEY"]
res = conn.ai.createDocumentIngest(
    data_source="s3",
    data_source_config={"aws_access_key": access, "aws_secret_key": sec},
    loader_config={"doc_id_field": "url", "content_field": "content"},
    file_format="json",
)

In [14]:
conn.ai.runDocumentIngest(
    res["load_job_id"],
    res["data_source_id"],
    "s3://tg-documentation/pytg_current/pytg_current.jsonl",
)

{'job_name': 'load_documents_content_json_b89acfebac9e4fb98efd20a49659808e',
 'job_id': 'GraphRAG_pytgdocs.load_documents_content_json_b89acfebac9e4fb98efd20a49659808e.stream.SupportAI_GraphRAG_pytgdocs_5698bff74d844534901cba9e1b3d55bf.1722466964295',
 'log_location': '/home/tigergraph/tigergraph/log/kafkaLoader/GraphRAG_pytgdocs.load_documents_content_json_b89acfebac9e4fb98efd20a49659808e.stream.SupportAI_GraphRAG_pytgdocs_5698bff74d844534901cba9e1b3d55bf.1722466964295'}

In [18]:
import httpx
import base64


def make_headers(conn: TigerGraphConnection):
    tkn = base64.b64encode(f"{conn.username}:{conn.password}".encode()).decode()
    headers = {"Authorization": f"Basic {tkn}"}
    return headers


httpx.get(
    "http://localhost:8001/GraphRAG_pytgdocs/consistency_status/graphrag",
    headers=make_headers(conn),
)
# conn.ai.forceConsistencyUpdate()

<Response [200 OK]>

In [16]:
asdf

NameError: name 'asdf' is not defined

In [30]:
for v in ["Document", "Content", "DocumentChunk", "Entity","ResolvedEntity"]:
# for v in ["ResolvedEntity"]:
    try:
        conn.delVertices(v)
    except:
        pass

import time

time.sleep(3)
conn.ai.runDocumentIngest(
    res["load_job_id"],
    res["data_source_id"],
    "s3://tg-documentation/pytg_current/pytg_current.jsonl",
)

{'job_name': 'load_documents_content_json_b89acfebac9e4fb98efd20a49659808e',
 'job_id': 'GraphRAG_pytgdocs.load_documents_content_json_b89acfebac9e4fb98efd20a49659808e.stream.SupportAI_GraphRAG_pytgdocs_5698bff74d844534901cba9e1b3d55bf.1722531204658',
 'log_location': '/home/tigergraph/tigergraph/log/kafkaLoader/GraphRAG_pytgdocs.load_documents_content_json_b89acfebac9e4fb98efd20a49659808e.stream.SupportAI_GraphRAG_pytgdocs_5698bff74d844534901cba9e1b3d55bf.1722531204658'}

In [ ]:
conn.gsql(f"""
USE GRAPH {conn.graphname}
DROP QUERY ResolveRelationships
""")

In [33]:
conn.graphname = "Cora"
conn.getToken()
for v in ["Community"]:
    # for v in ["ResolvedEntity"]:
    conn.delVertices(v)

KeyError: 'deleted_vertices'

In [ ]:
import base64
import json
import httpx
import logging

_ = logging.getLogger(__name__)


http_timeout = None


def make_headers(conn: TigerGraphConnection):
    if conn.apiToken is None or conn.apiToken == "":
        tkn = base64.b64encode(f"{conn.username}:{conn.password}".encode()).decode()
        headers = {"Authorization": f"Basic {tkn}"}
    else:
        headers = {"Authorization": f"Bearer {conn.apiToken}"}

    return headers


def check_vertex_exists(conn, id):
    headers = make_headers(conn)
    with httpx.Client(timeout=http_timeout) as client:
        res = client.get(
            f"{conn.restppUrl}/graph/{conn.graphname}/vertices/Entity/{id}",
            headers=headers,
        )

        res.raise_for_status()
    return res.json()


# r = check_vertex_exists(conn, "asdfTigergraphexception")
# print(json.dumps(r, indent=2), r["error"])
r = check_vertex_exists(conn, "Tigergraphexception")
print(json.dumps(r, indent=2), r["error"])
r["results"][0]["attributes"]["description"]

In [ ]:
def map_attrs(attributes: dict):
    # map attrs
    attrs = {}
    for k, v in attributes.items():
        if isinstance(v, tuple):
            attrs[k] = {"value": v[0], "op": v[1]}
        elif isinstance(v, dict):
            attrs[k] = {
                "value": {"keylist": list(v.keys()), "valuelist": list(v.values())}
            }
        else:
            attrs[k] = {"value": v}
    return attrs


def process_id(v_id: str):
    return v_id.replace(" ", "_").replace("/", "")


def a(vertex_id="Post /Requesttoken"):
    vertex_id = process_id(vertex_id)
    attributes = {  # attrs
        "description": ["test"],
        "epoch_added": int(time.time()),
    }

    vertex_id = vertex_id.replace(" ", "_")
    attrs = map_attrs(attributes)
    data = json.dumps({"vertices": {"Entity": {vertex_id: attrs}}})
    headers = make_headers(conn)
    with httpx.Client(timeout=http_timeout) as client:
        res = client.post(
            f"{conn.restppUrl}/graph/{conn.graphname}", data=data, headers=headers
        )

        res.raise_for_status()

    return res.json()


a()

In [ ]:
from urllib import parse

v_id = "Post_/Requesttoken"
v_id = process_id(v_id)
print(v_id)

r = check_vertex_exists(conn, v_id)
print(json.dumps(r, indent=2), r["error"])
r["results"][0]["attributes"]["description"]